We'll explore how to interact with a SQLite database in Python so we can start to incorporate databases into your data science workflow.

SQLite is a database that doesn't require a standalone server; it stores the entire database as a file on disk. This makes it ideal for working with larger data sets that can fit on disk but not in memory.

Python loads the entire data set we're working with into memory, making SQLite a compelling alternative for working with data sets larger than 8 gigabytes (which is roughly the amount of memory modern computers contain). The fact that we can contain an entire database in a single file makes them easy to share; some data sets are available online as SQLite database files (using the extension .db).

We can interact with a SQLite database in two main ways:

* Through the [sqlite3 Python module](https://docs.python.org/3/library/sqlite3.html)
* Through the [SQLite shell](https://sqlite.org/cli.html)

We'll focus on learning how to use the sqlite3 module to interact with the database.

In [1]:
import sqlite3
conn = sqlite3.connect("jobs.db")

Tuples are faster than lists, so they're helpful with larger databases and larger results sets.

In [2]:
cursor = conn.cursor()
query = "select * from recent_grads;"
cursor.execute(query)
results = cursor.fetchall()
print(results[0:2])

[(0, 1, 2419, 'PETROLEUM ENGINEERING', 'Engineering', 2339, 36, 2057, 282, 0.120564344, 1976, 1849, 270, 1207, 37, 0.018380527, 110000, 95000, 125000, 1534, 364, 193), (1, 2, 2416, 'MINING AND MINERAL ENGINEERING', 'Engineering', 756, 7, 679, 77, 0.10185185199999999, 640, 556, 170, 388, 85, 0.117241379, 75000, 55000, 90000, 350, 257, 50)]


So far, we've been running queries by creating a Cursor instance, and then calling the execute method on the instance. The SQLite library actually allows us to skip creating a Cursor altogether by using the execute method within the Connection object itself. SQLite will create a Cursor instance for us under the hood and run our query against the database, allowing us to skip a step.

In [3]:
conn = sqlite3.connect("jobs.db")
query = "select * from recent_grads;"
print(conn.execute(query).fetchall()[:2])

[(0, 1, 2419, 'PETROLEUM ENGINEERING', 'Engineering', 2339, 36, 2057, 282, 0.120564344, 1976, 1849, 270, 1207, 37, 0.018380527, 110000, 95000, 125000, 1534, 364, 193), (1, 2, 2416, 'MINING AND MINERAL ENGINEERING', 'Engineering', 756, 7, 679, 77, 0.10185185199999999, 640, 556, 170, 388, 85, 0.117241379, 75000, 55000, 90000, 350, 257, 50)]


To make it easier to work with large results sets, the Cursor class allows us to control the number of results we want to retrieve at any given time. To return a single result (as a tuple), we use the Cursor method fetchone(). To return n results, we use the Cursor method fetchmany().

Each Cursor instance contains an internal counter that updates every time we retrieve results. When we call the fetchone() method, the Cursor instance will return a single result, and then increment its internal counter by 1. This means that if we call fetchone() again, the Cursor instance will actually return the second tuple in the results set (and increment by 1 again).

The fetchmany() method takes in an integer (n) and returns the corresponding results, starting from the current position. It then increments the Cursor instance's counter by n. In the following code, we return the first two results using the fetchone() method, then the next five results using the fetchmany() method.

In [4]:
first_result = cursor.fetchone()
second_result = cursor.fetchone()
next_five_results = cursor.fetchmany(5)

Because SQLite restricts access to the database file when we're connected to a database, we need to close the connection when we're done working with it. Closing the connection allows other processes to access the database, which is important when you're in a production environment and working with other team members.

To close a connection to a database, use the Connection instance method close(). When we're working with multiple databases and multiple Connection instances, we want to make sure we call the close() method on the correct instance. After closing the connection, attempting to query the database using any linked Cursor instances will return the following error:

ProgrammingError: Cannot operate on a closed database.

In [5]:
conn.close()